# Frequency sweep
Now that we can control the function generator and the oscilloscope, we are ready to set up a frequency sweep. Start by connecting to both devices and creating log string for each. Concatenate the two log strings. We will be varying the function generator frequency and oscilloscope xscale throughout the project, so set these values to the string `'VARIABLE'` in the log file. We will log them later.

In [ ]:
import siglent_sds1104xe_oscope_solution as osc 
import siglent_sdg2082x_fgen_solution as fgen
from time import sleep
import numpy as np
import matplotlib.pyplot as plt
from simple_fft import simple_fft
import os

In [ ]:
osc_inst = osc.connect('192.168.1.4')
fgen_inst = fgen.connect('192.168.1.2')

osc.initialize(osc_inst)
inst_log = osc.create_log('VARIABLE', 0, 1, 0, 1, 'AUTO') + '\n'

fgen.initialize(fgen_inst)
fgen.set_amplitude(fgen_inst, 1, 5)
inst_log += fgen.create_log(1, 'SINE', 'VARIABLE', 5, 0) + '\n'

Now, we need a function that will set the frequency of the function generator and set the xscale of the oscilloscope to cover two periods per division. Put a sleep statement at the end of the function to ensure that both devices have time to be set properly.

In [ ]:
def set_frequency(frequency):
    # Set frequency of function generator
    fgen.set_frequency(fgen_inst, 1, frequency)
    # Set xscale of oscilloscope
    tperiod = 1 / frequency 
    xscale = tperiod / 2
    
    osc.set_xscale(osc_inst, xscale)

    sleep(0.1)

We will now set up a frequency sweep. Using the following parameters, append to the log file the information about the frequency sweep and create a list of frequencies. Change the output directory to your group name. We have written code to raise an error if the files already exist, so you don't overwrite another group's data.

In [ ]:
start_freq = 100 
end_freq = 30e3 
npoints = 50

frequencies = np.linspace(start_freq, end_freq, npoints)
log = inst_log + "# -------------- Frequency sweep settings -------------- #\n" 
log += f'start frequency: {start_freq} Hz\n' 
log += f'end frequency: {end_freq} Hz\n' 
log += f'number of points: {npoints}\n'


out_directory = '/home/ifirse05/data/sagi/group1/sweep_00/'
log_path = out_directory + 'freqsweep.log' 
# if os.path.exists(log_path):
#     raise FileExistsError(f'{log_path} already exists')
with open(log_path, 'w') as f:
    f.write(log)

Now, we will create a loop to iteratively set the frequency and save data. For each frequency in `frequencies`, first set the frequency, then capture data, then save the data. Don't forget to turn on the function generator output before taking data.

In [ ]:
fgen.set_channel_state(fgen_inst, 1, 'ON')

for index, frequency in enumerate(frequencies):
    set_frequency(frequency)
    tsample, time, voltage = osc.capture_data(osc_inst) 
    np.save(out_directory + f'freqsweep_{index:03d}.npy', [time, voltage])

fgen.set_channel_state(fgen_inst, 1, 'OFF')

# Analysis
We will now import each set of data, FFT the data, extract the amplitude at the desired frequency, and plot the amplitude versus frequency. As a sanity check, plot a few of your timestreams and FFTs with an indication of the value you have selected as the peak.

In [ ]:
index = 4

frequency = frequencies[index]
time, voltage = np.load(out_directory + f'freqsweep_{index:03d}.npy') 
tsample = time[1] - time[0]
f, y = simple_fft(tsample, voltage)
ix = np.argmin(abs(f - frequency))

fig, axs = plt.subplots(2, 1, figsize = [10, 5])
axs[1].set_xscale('log')
axs[0].plot(time, voltage) 
axs[1].plot(f, y)
axs[1].plot(f[ix], y[ix], 'xr')

In [ ]:
transmission = []
for index, frequency in enumerate(frequencies):
    time, voltage = np.load(out_directory + f'freqsweep_{index:03d}.npy') 
    tsample = time[1] - time[0]
    f, y = simple_fft(tsample, voltage)
    ix = np.argmin(abs(f - frequency))
    transmission.append(y[ix]/ 2.5) # Divide by the input voltage to get transmission

In [ ]:
fig, ax = plt.subplots()
ax.set(ylabel = 'Transmission', xlabel = 'Frequency (MHz)')
ax.plot(frequencies / 1e6, np.array(transmission))

# Cleaning up frequency sweep code
You have developed the code you need to perform a frequency sweep, and now you want to wrap the code into a single function to use for the rest of the lab. Write this function and move it to 'frequency_sweep.py'. Run the code from this notebook to test it.

In [ ]:
import siglent_sds1104xe_oscope_solution as osc 
import siglent_sdg2082x_fgen_solution as fgen
from freq_sweep import capture_freq_sweep

out_directory = '/home/ifirse05/data/sagi/group1/sweep_00/'
osc_inst = osc.connect('192.168.1.4')
fgen_inst = fgen.connect('192.168.1.2')

start_freq = 100 
end_freq = 1e6 
npoints = 200

capture_freq_sweep(out_directory, osc_inst, fgen_inst, start_freq, end_freq, npoints)